# はじめに

このノートではLightGBMを理解するために、LightGBMと同じくらい使用頻度の高いXGboostのアルゴリズムについても理解を深めておく。LightGBMの特徴については別のノートで取り上げる予定ではあるが、XGboostの利点を多く備えているのがLightGBMでもあるので、まずはXGboostを理解する。下記のmeanxaiの動画を参考にしている。

- [[MXML-11-07] Extreme Gradient Boosting (XGBoost) [7/9] - Approximate Algorithm for Split Finding](https://www.youtube.com/watch?v=AQOPXlxXF_0&list=PLgaemU2xZlTieGRRTAdDlHOgxr-bs6pfr&index=8)
- [[MXML-11-08] Extreme Gradient Boosting (XGBoost) [8/9] - Weighted Quantile Sketch for Split Finding](https://www.youtube.com/watch?v=ejUvX1L-yzE&list=PLgaemU2xZlTieGRRTAdDlHOgxr-bs6pfr&index=8)
- [[MXML-11-09] Extreme Gradient Boosting (XGBoost) [9/9] - Sparsity-aware Split Finding](https://www.youtube.com/watch?v=fALcIVr6zjY&list=PLgaemU2xZlTieGRRTAdDlHOgxr-bs6pfr&index=6)

今回はXGboostの下記のアルゴリズムを理解することを目的にする。

- Approximate Algorithm for Split Finding
- Weighted Quantile Sketch for Split Finding
- Sparsity-aware for Split Finding

## Approximate Algorithm

Exact Greedy Algorithmは(EGA)、特徴量のすべての分割点の候補を用いてノードを分割する、まさにGreedy(貪欲な)アルゴリズム。そのため、最適な分割点を正確に計算できる一方で、ビックデータに対しては、計算が完了しない問題が発生してしまう。そこで、Approximate Algorithm(AA)を用いて近似的に最適な分割点を求めることが行われる。実行手順は下記の通りである。

1. 前の予測値をもとに残差を計算
2. 各特徴量(x1,...,xn)に対して、昇順の並び替えを行う(sorted x1: sx1)。
3. sx1を1/εという幅でバケットに分割する。ε=0.2であればバケットは5つできる。
4. 各バケットに同じデータ数でデータをアサインする。パーセンタイルの考え方。
5. 各バケットにスレッドを割り当て、並列処理を行う。各スレッドはバケット内のデータからGainを計算する。各バケットでベストなGainを探索する。
6. 各バケットで計算されたベストなGainから、最も良いものを選択し、最適な分割点とする。
7. さらに、各特徴量(x1,...,xn)の中から最大のGainを得られる分割点を最終的な分割点とする。

Approximate AlgorithmにはGlobal variantとLocal variantという2種類の方法がある。Global variantでは最初は100バケットあったとすると、分岐後では(50/50)バケット、さらに分岐後は(20/30)バケット、さらにさらに分岐後は(10/10)バケットというように、分岐が行われるたびに最初に確保したバケットを減らしていく。Local variantでは分岐後に1/εという幅で再度バケットを確保する。

下記の図ではバケットを5つに分割し、各バケットにスレッドを割り当て、Gainを計算している。3つ目のバケットがこの場合、ベストな分割点である。なぜなら、緑の線で分岐した際に、左右で残差の方向が反対になって偏りが発生しているため、Gainが大きくなりやすいため。

<img src='./BoostingAA.png'>

下記が実際に実行した結果である。Exact Greedy Algorithmの最適な分割点は0.497で5秒ほどかかっている。一方で、Approximate Algorithmの最適な分割点は0.496で、1秒かかっておらず、高速化しても同じ分割点が得られている。

In [47]:
# Exact greedy algorithm: 
# split point = 0.497
# gain = 81396.288
# running time = 5.37 seconds

# Approximate Algorithm: 
# split points = [0.083 0.344 0.496 0.659 0.916]
# gain = [2565.74 515.64 3816.13 420.49 2499.57]
# running time = 0.30 seconds
# number of data in blocks = [10000, 10000, 10000, 10000, 10000]

参考までにコードもメモしておくが、このままでは実行できず、MyXGBRegressorクラス内でApproximate Algorithmを実行する必要があるとのこと。

In [ ]:
# import numpy as np
# import time
# 
# # Create training data to cet .
# def nonlinear_data(n, s):
#   rtn_x, rtn_y = [], [] 
#   for i in range(n):
#     x = np.random. random()
#     y = 2.0 * np.sin(2.0 * np.pi * x) + np.random.normal(0.0, s) + 3.0
#     rtn_x.append(x)
#     rtn_y.append(y)
#   return np.array(rtn_x).reshape(-1,1), np.array(rtn_y)
#   
# x, y = nonlinear_data(n=50000, s=0.5)
# 
# from MyXGBoostRegressor import MyXGBRegressor
# 
# # Exact Greedy Algorithm (EGA)
# start_time = time.time()
# my_model = MyXGBRegressor(n_estimators=1, max_depth=1,base_score=y.mean())
# my_model.fit(x, y)
# e = my_model.models[0].estimator2
# print('\nExact greedy algorithm: ')
# print('split point =', np.round(e['split_point'], 3))
# print('gain =', np.round(e['gain'], 3))
# print('running time = {:.2f} seconds'.format(time.time() - start_time))

# ------------------------------------------------------------------------------------------------
# > MyXGBRegressor is a class implemented with EGA. To implement this properly,
# > you need to implement the Approximate Algorithm inside the MyXGBRegressor.
# ------------------------------------------------------------------------------------------------
# # 2.Approximate Algorithm (AA). :
# from multiprocessing.pool import Pool
# 
# def find_split_point(x, y):
#   my_model = MyXGBRegressor(n_estimators=1,max_depth=1,base_score=y.mean())
#   my_model.fit(x, y)
#   e = my_model.models[0].estimator2
#   return [e['split_point'], e['gain']]
# 
# # Divide the data into five parts and allocate 20% of the data to |
# # each part. (e=0.2)
# c_point = np.percentile(x, [20, 40, 60, 80, 100])
# # maps the data into buckets split by c_point
# l_bound = -np.inf
# x_block, y_block = [], []
# for p in c_point:
#   idx = np.where(np.logical_and(x > l_bound, x <= p))[0]
#   x_block. append (x[ idx])
#   y_block. append (y[idx])
#   l_bound = p
# start_time = time.time()
# mp = Pool(5) # 5 processes
# args = [[ax, ay] for ax, ay in zip(x_block, y_block)]
# ret = mp.starmap_async(find_split_point, args)
# mp.close()
# mp.join()
# print('\nApproximate Algorithm: ')
# print('split point =', np.array(ret.get())[:, 0].round(3))
# print('gain =', np.array(ret.get())[:, 1].round(2))
# print('running time = {:.2f} seconds'.format(time.time() - start_time))
# print ('number of data in blocks =', [len(a) for a in x_block])

## Weighted Quantile Sketch for Split Finding

Approximate Algorithmでは、各バケットに同じデータ量をアサインする。その点を重みをつけて分割するのが、Weighted Quantile Sketchアルゴリズム。そもそも、Sketchアルゴリズムは、大規模なデータセットやリアルタイムデータ処理の環境で、メモリや計算リソースを節約しながら、データの要約や基本的な統計情報を得るため方法として利用される。

ここでの重みは回帰の場合、損失関数がMSEだとテイラー展開の2次微分$h$は$1$なので、$\sum n$となる。一方。分類の場合はクロスエントロピーを利用するため、$h=p(1-p)$となり、複雑になる。そのため、各バケットの$h$の合計が同じになるようにバケットを分割する。基本的には下記の通り実行される。


1. $k$個の特徴量がある場合、$k$番目の特徴量と$h$の組を考える。$h^{(2)}_{i}=\hat{y_{i}}^{(1)}(1-\hat{y_{i}}^{(1)})$である。

$$
D_{k} = {(x_{1,k},h_{1}), (x_{2,k},h_{2}),...,(x_{n,k},h_{n})}
$$

2. $k$番目の特徴量を$h$を利用して、$m$分割されたバケットにデータをアサインする。

$$
S_{k} = \{s_{k,1},s_{k,2},s_{k,3},...,s_{k,m}\}
$$

3. 標準化されたランク関数を使用する。これは$h$の割合を表す。$|r_{k}(s_{k,j}) - r_{k}(s_{k,j+1})| < \epsilon$の時、新しくバケットを作成する。

$$
r_{k}(z) = \frac{\sum_{(x,h) \in D_{k}, x<z}}{\sum_{(x,h) \in D_{k}}}
$$



なぜ$h$が重みとして機能するのか。これは、$h=p(1-p)$が関係している。新しく予測された$p$は0や1に近いほど、予測の信頼度合いが高いことを意味する。例えば$0.1*(0.9)=0.09,0.5*(0.5)=0.25$の通りである。これを重みとして、バケットの合計を揃えると、予測の信頼度合いが高いものは、バケット内に沢山集めることができ、予測の信頼度合いが低い物は、それだけで$h$が大きくなるので、バケットがそれだけになることで、信頼度合いが低いものを優先的に調べたいというお気持ちである。xgboostパッケージの`tree_method`引数を`exact`から`approx`に変更することで利用可能。

In [ ]:
# model = XGBClassifier(
#   n_estimators = TREES,
#   max_depth = DEPTH,
#   learning_rate = ETA,
#   gamma = GANA, |
#   reg_lambda = LAMB,
#   base_score = 0.5,
#   tree_method = 'exact'
#   )

## Sparsity-aware Split Finding

Sparsity-aware Split Findingアルゴリズムは、現実のデータがスパースであることを考慮するためのアルゴリズム。ここでのスパースとは欠損値、0、ワンホットエンコーディングによるものを指す。Sparsity-aware Split Findingアルゴリズムは下記の手順で実行される。

1. ノードの分割を行う際に、スパースなデータ以外を使用して分割点をみつける
2. スパースなデータを左の葉に入れたときのGain、右の葉に入れたときのGainを計算する
3. Gainが大きい方にスパースなデータを割り当てる。スパースなデータはこれに沿って分岐される。
4. 新しい予測のデータにスパースなデータが含まれていても、上記の通り、分岐される。